<a href="https://colab.research.google.com/github/j-walker-red/shoes/blob/main/resize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Configuración
alto = 600
ancho = round(350/220*alto)  # La proporción es 350:220

# Importamos librerías
import cv2
from google.colab import files
import numpy as np
import os
import zipfile

# Función que detecta el zapato en la imagen y cambia su tamaño
def resize_and_detect_shoe(image_path, output_path, target_width=ancho, target_height=alto):
    # Leer la imagen con OpenCV
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"No se pudo abrir la imagen: {image_path}")

    # Convertir a escala de grises
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Aplicar umbral adaptativo para separar el zapato del fondo
    _, thresh = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY_INV)

    # Encontrar contornos en la imagen
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Seleccionar el contorno más grande (asumiendo que es el zapato)
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(largest_contour)
    else:
        raise ValueError("No se detectó ningún contorno en la imagen")

    # Extraer el área del zapato
    shoe = image[y:y+h, x:x+w]

    # Redimensionar manteniendo proporciones
    shoe_h, shoe_w, _ = shoe.shape
    aspect_ratio = shoe_w / shoe_h
    if aspect_ratio > (target_width / target_height):
        new_width = target_width
        new_height = int(target_width / aspect_ratio)
    else:
        new_height = target_height
        new_width = int(target_height * aspect_ratio)

    resized_shoe = cv2.resize(shoe, (new_width, new_height), interpolation=cv2.INTER_LANCZOS4)

    # Crear un lienzo blanco del tamaño deseado
    canvas = np.ones((target_height, target_width, 3), dtype=np.uint8) * 255

    # Calcular los offsets para centrar
    x_offset = (target_width - new_width) // 2
    y_offset = (target_height - new_height) // 2

    # Pegar el zapato redimensionado en el lienzo blanco
    canvas[y_offset:y_offset+new_height, x_offset:x_offset+new_width] = resized_shoe

    # Guardar el resultado
    cv2.imwrite(output_path, canvas)

# Procesa las imágenes en el fichero comprimido
def process_images_in_zip(zip_path, output_zip_path, target_width=ancho, target_height=alto):
    # Crear un directorio temporal para extraer las imágenes
    temp_input_dir = '/content/temp_input_images'
    temp_output_dir = '/content/temp_output_images'
    os.makedirs(temp_input_dir, exist_ok=True)
    os.makedirs(temp_output_dir, exist_ok=True)

    # Extraer el archivo zip
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(temp_input_dir)

    # Procesar cada archivo en la carpeta extraída
    for filename in os.listdir(temp_input_dir):
        # Verificar si es un archivo de imagen válido
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            input_path = os.path.join(temp_input_dir, filename)
            output_filename = os.path.splitext(filename)[0] + "_redim.jpg"
            output_path = os.path.join(temp_output_dir, output_filename)

            try:
                # Procesar la imagen
                resize_and_detect_shoe(input_path, output_path, target_width, target_height)
                print(f"Procesado: {filename} -> {output_filename}")
            except Exception as e:
                print(f"Error procesando {filename}: {e}")

    # Crear un archivo zip con las imágenes transformadas
    with zipfile.ZipFile(output_zip_path, 'w') as zipf:
        for root, _, files in os.walk(temp_output_dir):
            for file in files:
                zipf.write(os.path.join(root, file), file)

    # Limpiar los directorios temporales
    import shutil
    shutil.rmtree(temp_input_dir)
    shutil.rmtree(temp_output_dir)

# Procesar las imágenes
# Subir el archivo zip desde el disco duro
uploaded = files.upload()

# Procesar el archivo zip subido
for filename in uploaded.keys():
    process_images_in_zip(filename, '/content/output_images.zip')
    files.download('/content/output_images.zip')